# Demo Balance de Energía

**Python (Pandas)**
*   Simplicidad
*   Familiaridad
*   Rendimiento para pequeños volumenes
*   Bajo costo (open source)

**Colab**

* Implementa procesos automáticos de respaldo, mediante scripts que sincronicen con Google Drive, además de que permite la mitigación de errores mediante la implementación de notebooks (paso a paso)
* Entorno de desarrollo serverless eficiente, entendible y fácil de manipular
* Completa integración con python

**Escalabilidad**

Integración con Dask: computación y procesamiento de datos distribuída (paralización) para medianos volumenes de datos.





Importe de librerías:

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
import os ## Librería del SO para cargar y leer archivos

Definición e inicialización de variables necesarias:

In [2]:
fecha_inicio="2024-07-09" ## Para que se pueda modificar la fecha del proceso en caso de que se requiera
fecha_fin="2024-07-20"
id_archivo_balance_drive="1EDzz9Ep9VzHwVrHNy9Z_dL5K8D0r4oKv" #ID Archivo con errores!!
id_archivo_balance_drive_cor="1gsiY85tRa5_3QLtjRvEAz403tTgkSLDn" #ID Archivo sin error
api_key_1="lbmgOxpdzM9ffbXmvF3UyPhxwEHFf53wFSDk8ddqEQeor9eXetbGiYqlUkKPHIBL"
api_key_2="QlgVYk5LeVCWCD6BoNoWFkn1lhtUC3vPuFpWBuNkpRo6R4trrNnyqux6LTTn8I4n"

# Extracción

**Para procesar el archivo con errores en sus valores:**

In [ ]:
download_url = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive}'
local_file_path = '/tmp/balances3.xlsx'

## "response" es un objeto que contiene los datos en binario que se desean escribir en el archivo
response = requests.get(download_url)
with open(local_file_path, 'wb') as file: ## Abre un archivo en modo escritura binaria ('wb'). El gestor (context manager) asegura que el archivo se cierre automáticamente cuando se salga del bloque "with"
    file.write(response.content) ## Este método escribe datos en binario en el archivo abierto "file".

**Para procesar el archivo sin errores en sus valores:**

In [3]:
download_url_cor = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive_cor}'
local_file_path = '/tmp/balances4.xlsx'
response = requests.get(download_url_cor)
with open(local_file_path, 'wb') as file:
    file.write(response.content)

Capacidad real de generadoras

In [6]:
dfArchivoCapacidad = pd.read_excel(local_file_path,engine='openpyxl')
dfArchivoCapacidad.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Programa de Generación por Planta,NaN,NaN,NaN,NaN
1,NaN,Empresa Generadora ACME S.A. E.S.P.,NaN,NaN,NaN,NaN
2,NaN,Despacho Real 10/07/2024,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,FECHA,PLANTA,GENERADOR,CAPACIDAD (Kwh),CODIGO
5,NaN,2024-07-10 00:00:00,TERMOZIPA,2,33000,ZPA2
6,NaN,2024-07-10 01:00:00,TERMOZIPA,2,33000,ZPA2
7,NaN,2024-07-10 02:00:00,TERMOZIPA,2,33000,ZPA2
8,NaN,2024-07-10 03:00:00,TERMOZIPA,2,33000,ZPA2
9,NaN,2024-07-10 04:00:00,TERMOZIPA,2,20834,ZPA2


In [4]:
## Para obtener desde el simem los precios. el datasetId al final de la url hay que buscarlo en simem Precios
url = f"https://www.simem.co/backend-files/api/PublicData?startDate={fecha_inicio}&endDate={fecha_fin}&datasetId=96D56E"

response = requests.get(url)
json_precios = json.loads(response.text)

In [5]:
dfPreciosBolsa = pd.DataFrame(json_precios['result']['records'])
print(dfPreciosBolsa)


   CodigoVariable       Fecha CodigoDuracion UnidadMedida Version     Valor
0       PPBOGReal  2024-07-20            P1D      COP/kWh     TXF  321.5839
1            PPBO  2024-07-20            P1D      COP/kWh     TXF  322.3219
2       PPBOGReal  2024-07-20            P1D      COP/kWh     TX2  315.5043
3            PPBO  2024-07-20            P1D      COP/kWh     TX2  316.2516
4       PPBOGReal  2024-07-20            P1D      COP/kWh     TX1  315.4548
..            ...         ...            ...          ...     ...       ...
79           PPBO  2024-07-09            P1D      COP/kWh     TXR  243.7167
80      PPBOGReal  2024-07-09            P1D      COP/kWh     TXR  254.6227
81           PPBO  2024-07-09            P1D      COP/kWh     TX2  237.6496
82      PPBOGReal  2024-07-09            P1D      COP/kWh     TXF  254.6253
83           PPBO  2024-07-09            P1D      COP/kWh     TXF  243.7194

[84 rows x 6 columns]


In [7]:
## Para obtener desde SIMEM el compromiso o los despachos por unidad.
url = f"https://www.simem.co/backend-files/api/PublicData?startdate={fecha_inicio}&enddate={fecha_fin}&datasetId=ff027b"
response = requests.get(url)
json_precios = json.loads(response.text)

In [8]:
dfDespachosUnidades = pd.DataFrame(json_precios['result']['records'])
print(dfDespachosUnidades.head(10))

             FechaHora CodigoDuracion CodigoPlanta     Valor
0  2024-07-20 23:00:00           PT1H         MND1     420.0
1  2024-07-20 23:00:00           PT1H         MNC1    1200.0
2  2024-07-20 23:00:00           PT1H         MIR1    2470.0
3  2024-07-20 23:00:00           PT1H         MATA       0.0
4  2024-07-20 23:00:00           PT1H         LTSJ  204000.0
5  2024-07-20 23:00:00           PT1H         LPLO     950.0
6  2024-07-20 23:00:00           PT1H         LMR1       0.0
7  2024-07-20 23:00:00           PT1H         LCS1       0.0
8  2024-07-20 23:00:00           PT1H         LBR1    2000.0
9  2024-07-20 23:00:00           PT1H         JPR1       0.0


# Transformación

Ahora, es necesario filtrar la columna código de planta en el DF de despachos para que permanezcan solo los registros que contengan ZPA2, ZPA3, ZPA4, ZPA5, GVIO, QUI1, CHVR

In [9]:
listaPlantas = ['ZPA2','ZPA3','ZPA4','ZPA5','GVIO','QUI1']

In [10]:
#Función para filtrar un dataframe por código planta.
#Recibe el dataframe implicado y una lista que contiene los código planta que se quiere
#Retorna el dataframe con los registros cuyo valor es True (filtro interno con el isin)

def filtroCodigoPlanta(df, codigos):
    return df[df['CodigoPlanta'].isin(codigos)]


In [11]:
dfCapacidadDespachosAcme = filtroCodigoPlanta(dfDespachosUnidades, listaPlantas)

# Resetear los índices del dataframe
dfCapacidadDespachosAcme = dfCapacidadDespachosAcme.reset_index(drop=True)

print(dfCapacidadDespachosAcme)

                FechaHora CodigoDuracion CodigoPlanta      Valor
0     2024-07-20 23:00:00           PT1H         ZPA5    31000.0
1     2024-07-20 23:00:00           PT1H         ZPA4        0.0
2     2024-07-20 23:00:00           PT1H         ZPA3        0.0
3     2024-07-20 23:00:00           PT1H         ZPA2        0.0
4     2024-07-20 22:00:00           PT1H         GVIO  1220000.0
...                   ...            ...          ...        ...
1723  2024-07-09 03:00:00           PT1H         ZPA5    31000.0
1724  2024-07-09 02:00:00           PT1H         ZPA2    17000.0
1725  2024-07-09 02:00:00           PT1H         ZPA3    31000.0
1726  2024-07-09 02:00:00           PT1H         ZPA4    31000.0
1727  2024-07-09 02:00:00           PT1H         ZPA5    31000.0

[1728 rows x 4 columns]


A continuación se renombran las columnas del dataframe de la capacidad y se nomalizan los datos a tipo string

In [12]:
# Renombramiento de columnas
colRenom = {'Unnamed: 0':'Useless',
            'Unnamed: 1':'Fecha',
            'Unnamed: 2':'Planta',
            'Unnamed: 3':'Generador',
            'Unnamed: 4':'Capacidad',
            'Unnamed: 5':'Codigo'}

dfArchivoCapacidad = dfArchivoCapacidad.rename(columns=colRenom).astype(str)
print(dfArchivoCapacidad.head(10))

  Useless                                Fecha     Planta  Generador  \
0     nan    Programa de Generación por Planta        nan        nan   
1     nan  Empresa Generadora ACME S.A. E.S.P.        nan        nan   
2     nan             Despacho Real 10/07/2024        nan        nan   
3     nan                                  nan        nan        nan   
4     nan                                FECHA     PLANTA  GENERADOR   
5     nan                  2024-07-10 00:00:00  TERMOZIPA          2   
6     nan                  2024-07-10 01:00:00  TERMOZIPA          2   
7     nan                  2024-07-10 02:00:00  TERMOZIPA          2   
8     nan                  2024-07-10 03:00:00  TERMOZIPA          2   
9     nan                  2024-07-10 04:00:00  TERMOZIPA          2   

         Capacidad  Codigo  
0              nan     nan  
1              nan     nan  
2              nan     nan  
3              nan     nan  
4  CAPACIDAD (Kwh)  CODIGO  
5            33000    ZPA2  
6   

Se elimina la información innecesaria del archivo de capacidad real

In [13]:
# Eliminar filas innecesarias antes de la fila 4 (donde están los encabezados reales)
dfArchivoCapacidad = dfArchivoCapacidad.drop([0, 1, 2, 3, 4])

# Eliminar la columna 'Useless' que no es necesaria
dfArchivoCapacidad = dfArchivoCapacidad.drop(columns=['Useless'])

# Resetear los índices del dataframe
dfArchivoCapacidad = dfArchivoCapacidad.reset_index(drop=True)

# Mostrar el dataframe limpio
print(dfArchivoCapacidad)

                   Fecha     Planta Generador Capacidad Codigo
0    2024-07-10 00:00:00  TERMOZIPA         2     33000   ZPA2
1    2024-07-10 01:00:00  TERMOZIPA         2     33000   ZPA2
2    2024-07-10 02:00:00  TERMOZIPA         2     33000   ZPA2
3    2024-07-10 03:00:00  TERMOZIPA         2     33000   ZPA2
4    2024-07-10 04:00:00  TERMOZIPA         2     20834   ZPA2
..                   ...        ...       ...       ...    ...
139  2024-07-10 19:00:00     QUIMBO         1    284650   QUI1
140  2024-07-10 20:00:00     QUIMBO         1    276992   QUI1
141  2024-07-10 21:00:00     QUIMBO         1    340000   QUI1
142  2024-07-10 22:00:00     QUIMBO         1    340000   QUI1
143  2024-07-10 23:00:00     QUIMBO         1    340000   QUI1

[144 rows x 5 columns]


Se hace validación de formatos y verificación de los valores contenidos en el dataframe de la capacidad real de las generadoras.

In [14]:
def validar_fecha(fecha):
    patron = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$'
    return bool(re.match(patron, fecha))

def validar_generador(generador):
    try:
        int(generador)
        return True
    except ValueError:
        return False

def validar_capacidad(capacidad):
    try:
        int(capacidad)
        if int(capacidad) > 0: return True
        else: return False
    except ValueError:
        return False



# Se aplican las funciones de validación a los valores de las columnas correspondientes

dfArchivoCapacidad['Fecha_valida'] = dfArchivoCapacidad['Fecha'].apply(validar_fecha)
dfArchivoCapacidad['Generador_valido'] = dfArchivoCapacidad['Generador'].apply(validar_generador)
dfArchivoCapacidad['Capacidad_valida'] = dfArchivoCapacidad['Capacidad'].apply(validar_capacidad)

# Se filtran las filas que no cumplen con el formato de fecha y las condiciones del generador
fechas_invalidas = dfArchivoCapacidad[~dfArchivoCapacidad['Fecha_valida']]
generadores_invalidos = dfArchivoCapacidad[~dfArchivoCapacidad['Generador_valido']]
capacidades_invalidas = dfArchivoCapacidad[~dfArchivoCapacidad['Capacidad_valida']]

# Captura de error y reporte al usuario
if not fechas_invalidas.empty:
    print("Fechas con formato incorrecto. Por favor revisar el archivo")
    print(fechas_invalidas)

elif not generadores_invalidos.empty:
    print("Generadores con formato incorrecto. Por favor revisar el archivo")
    print(generadores_invalidos)

elif not capacidades_invalidas.empty:
    print("Capacidades invalidas. Por favor revisar el archivo")
    print(capacidades_invalidas)

else:

    # Se filtran las filas que cumplen con todas las validaciones
    df_validado = dfArchivoCapacidad[(dfArchivoCapacidad['Fecha_valida']) &
                                    (dfArchivoCapacidad['Generador_valido']) &
                                    (dfArchivoCapacidad['Fecha'] >= fecha_inicio) &
                                    (dfArchivoCapacidad['Fecha'] <= fecha_fin)]

    # Se procede a dividir la columna Fecha y crear el dataframe limpio
    df_validado['Año'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.year
    df_validado['Mes'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.month
    df_validado['Día'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.day
    df_validado['Hora'] = pd.to_datetime(dfArchivoCapacidad['Fecha']).dt.hour

    # Se eliminan las columnas de validación
    df_validado = df_validado.drop(columns=['Fecha_valida', 'Generador_valido'])

    # Selección de las columnas necesarias
    dfArchivoCapacidadLimpio = df_validado[['Año', 'Mes', 'Día', 'Hora', 'Codigo', 'Capacidad']]

    print(dfArchivoCapacidadLimpio)

      Año  Mes  Día  Hora Codigo Capacidad
0    2024    7   10     0   ZPA2     33000
1    2024    7   10     1   ZPA2     33000
2    2024    7   10     2   ZPA2     33000
3    2024    7   10     3   ZPA2     33000
4    2024    7   10     4   ZPA2     20834
..    ...  ...  ...   ...    ...       ...
139  2024    7   10    19   QUI1    284650
140  2024    7   10    20   QUI1    276992
141  2024    7   10    21   QUI1    340000
142  2024    7   10    22   QUI1    340000
143  2024    7   10    23   QUI1    340000

[144 rows x 6 columns]


Separación de la fecha, renombramiento y extracción de las columnas necesarias en el dataframe de la capacidad que XM solicita

In [15]:
dfCapacidadDespachosAcme['dia_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.day
dfCapacidadDespachosAcme['mes_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.month
dfCapacidadDespachosAcme['anio_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.year
dfCapacidadDespachosAcme['hora_despacho'] = pd.to_datetime(dfCapacidadDespachosAcme['FechaHora']).dt.hour

dfCapacidadDespachosAcme = dfCapacidadDespachosAcme.rename(columns={'Valor':'capacidad', 'CodigoPlanta':'codigo'})
dfDespachosAcmeTransformado = dfCapacidadDespachosAcme[['dia_despacho', 'mes_despacho', 'anio_despacho', 'hora_despacho', 'capacidad', 'codigo']]


print(dfDespachosAcmeTransformado)

      dia_despacho  mes_despacho  anio_despacho  hora_despacho  capacidad  \
0               20             7           2024             23    31000.0   
1               20             7           2024             23        0.0   
2               20             7           2024             23        0.0   
3               20             7           2024             23        0.0   
4               20             7           2024             22  1220000.0   
...            ...           ...            ...            ...        ...   
1723             9             7           2024              3    31000.0   
1724             9             7           2024              2    17000.0   
1725             9             7           2024              2    31000.0   
1726             9             7           2024              2    31000.0   
1727             9             7           2024              2    31000.0   

     codigo  
0      ZPA5  
1      ZPA4  
2      ZPA3  
3      ZPA2  
4    

Cálculo del balance de energía

In [16]:
# Une los DataFrames por las columnas especificadas
dfFusionado = pd.merge(dfArchivoCapacidadLimpio, dfDespachosAcmeTransformado,
                    left_on=['Codigo', 'Año', 'Mes', 'Día', 'Hora'],
                    right_on=['codigo', 'anio_despacho', 'mes_despacho', 'dia_despacho', 'hora_despacho'])

# Calcular el balance disponible horario
dfFusionado['balance_disponible_horario'] = dfFusionado['Capacidad'].astype(int) - dfFusionado['capacidad'].astype(int)

# Agrupa, calcula la suma y resetea el index
dfBalanceConsolidado = dfFusionado.groupby(['Año', 'Mes', 'Día', 'Codigo'])['balance_disponible_horario'].sum().reset_index()
dfBalanceConsolidado.rename(columns={'balance_disponible_horario': 'consolidado_planta'}, inplace=True)

print(dfBalanceConsolidado)

    Año  Mes  Día Codigo  consolidado_planta
0  2024    7   10   GVIO            11506280
1  2024    7   10   QUI1             3607414
2  2024    7   10   ZPA2              201706
3  2024    7   10   ZPA3             -280652
4  2024    7   10   ZPA4             -203312
5  2024    7   10   ZPA5             -362060


Cálculo del precio del balance y se agrega la columna de la operación a realizar según el resultado

In [17]:
# Filtra los precios de bolsa por las condiciones especificadas
dfPreciosFiltrados = dfPreciosBolsa[(dfPreciosBolsa['CodigoVariable'] == 'PPBOGReal') & (dfPreciosBolsa['Version'] == 'TXR')]

# Crea columnas de año, mes y día en dfPreciosFiltrados
dfPreciosFiltrados['Año'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.year
dfPreciosFiltrados['Mes'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.month
dfPreciosFiltrados['Día'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.day

# Une los DataFrames por las columnas año, mes y dia
dfUnido = pd.merge(dfBalanceConsolidado, dfPreciosFiltrados, on=['Año', 'Mes', 'Día'], how='inner')

# Calcular Compromisos_MCOP
dfUnido['Compromisos_MCOP'] = (dfUnido['consolidado_planta'] * dfUnido['Valor']) / 1000

# Seleccionar las columnas deseadas
dfReporteCompraVentaEnergiaAcme = dfUnido[['Año', 'Mes', 'Día', 'Codigo', 'consolidado_planta', 'Compromisos_MCOP']]

# Crea la nueva columna de Operación
# Si los compromisos son menores a 0, hay que comprar energía
dfReporteCompraVentaEnergiaAcme['Operacion'] = np.where(dfReporteCompraVentaEnergiaAcme['Compromisos_MCOP'] < 0, 'Comprar', 'Vender')

dfReporteCompraVentaEnergiaAcme.rename(columns={'Año': 'Anio', 'Día': 'Dia'}, inplace=True)

print(dfReporteCompraVentaEnergiaAcme)


   Anio  Mes  Dia Codigo  consolidado_planta  Compromisos_MCOP Operacion
0  2024    7   10   GVIO            11506280      2.528576e+06    Vender
1  2024    7   10   QUI1             3607414      7.927516e+05    Vender
2  2024    7   10   ZPA2              201706      4.432614e+04    Vender
3  2024    7   10   ZPA3             -280652     -6.167502e+04   Comprar
4  2024    7   10   ZPA4             -203312     -4.467907e+04   Comprar
5  2024    7   10   ZPA5             -362060     -7.956493e+04   Comprar


<ipython-input-17-3e4a7e0246ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPreciosFiltrados['Año'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.year
<ipython-input-17-3e4a7e0246ae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPreciosFiltrados['Mes'] = pd.to_datetime(dfPreciosFiltrados['Fecha']).dt.month
<ipython-input-17-3e4a7e0246ae>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

# Carga

En las siguientes líneas de código se almacena localmente en el notebook una carpeta que contendrá el archivo

In [18]:
ruta_carpeta = 'tmp'
os.makedirs(ruta_carpeta, exist_ok=True)
dfReporteCompraVentaEnergiaAcme.to_csv(f'{ruta_carpeta}//dfReporteCompraVentaEnergiaAcme.csv', index=False)

Inicio de sesión en la plataforma con la API y obtención del token para cargar el archivo

In [19]:
file_upload_access_token = ""
file_upload_account_id = ""
params = {'key1': api_key_1, 'key2': api_key_2}
response=requests.get("https://fastupload.io/api/v2/authorize", params)
json_response = json.loads(response.text)
try:
  file_upload_access_token = json_response["data"]["access_token"]
  file_upload_account_id = json_response["data"]["account_id"]
except:
  print("Error autenticando y autorizando en el servicio remoto de carga de archivos.")

In [20]:
print(file_upload_access_token)

ns6MxyjirZ25zT7SkuIhCAeGHiwFfb24JEpF64gTQ9ZyLMBedUZm3cr90q74IkIIl5sOpZV2XNciAQOFWXjd1xAlI8wn3OJbp6W0n2vMyl6HRVxtOKW5cKaKvvT5HG0Z


Finalmente la conexión a la API por medio de las Key y carga del archivo (post)

In [21]:
upload_folder_id = ""
json_response=""
with open("tmp/dfReporteCompraVentaEnergiaAcme.csv", "rb") as archivo:
  files = {"upload_file": (archivo.name, archivo)}  # Create a dictionary for the file upload

  data = {
    "access_token": file_upload_access_token,
    "account_id": file_upload_account_id,
    "folder_id": upload_folder_id  # Include folder_id if provided
  }

  response = requests.post("https://fastupload.io/api/v2/file/upload", files=files, data=data)

print(response.text)

{"response":"File uploaded","data":[{"name":"dfReporteCompraVentaEnergiaAcme (2).csv","size":342,"type":"application\/octet-stream","error":null,"url":"https:\/\/fastupload.io\/b30524259d2ae95d","delete_url":"https:\/\/fastupload.io\/b30524259d2ae95d~d?c380831246c9137e472c6bd5139dbb9a","info_url":"https:\/\/fastupload.io\/b30524259d2ae95d~i","delete_type":"DELETE","delete_hash":"c380831246c9137e472c6bd5139dbb9a","hash":"94f88bd605991fa72ea7751ae465407a","stats_url":"https:\/\/fastupload.io\/b30524259d2ae95d~s","short_url":"b30524259d2ae95d","file_id":"664630","unique_hash":"f64ea4667ae63850a8fd326fff9d915f9d46642a7a996a6170d00f47d8ce11ad","url_html":"&lt;a href=&quot;https:\/\/fastupload.io\/b30524259d2ae95d&quot; target=&quot;_blank&quot; title=&quot;View image on Fastupload.io&quot;&gt;View dfReporteCompraVentaEnergiaAcme (2).csv on Fastupload.io&lt;\/a&gt;","url_bbcode":"[url]https:\/\/fastupload.io\/b30524259d2ae95d[\/url]","success_result_html":""}],"_status":"success","_datetime"